# Оптимизация выполнения кода, векторизация, Numba

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. "Оптимизация выполнения кода, векторизация, Numba"
* IPython Cookbook, Second Edition (2018), глава 4
* https://ipython-books.github.io/43-profiling-your-code-line-by-line-with-line_profiler/
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html
* https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

## Задачи для совместного разбора

In [3]:
!pip install git+https://github.com/pyutils/line_profiler.git
!pip install line_profiler

  Cloning https://github.com/pyutils/line_profiler.git to c:\users\payrav\appdata\local\temp\pip-req-build-c1yockuo


  Running command git clone -q https://github.com/pyutils/line_profiler.git 'C:\Users\Payrav\AppData\Local\Temp\pip-req-build-c1yockuo'
  ERROR: Error [WinError 2] Не удается найти указанный файл while executing command git clone -q https://github.com/pyutils/line_profiler.git 'C:\Users\Payrav\AppData\Local\Temp\pip-req-build-c1yockuo'
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [4]:
import numpy as np
from numba import njit

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [3]:
np.random.randint(1,1_000,1_000_000)

array([273,  52, 769, ..., 524, 955, 232])

In [4]:
A = np.random.randint(0, 1000, size=(1000000))
A

def f1(A):
    acc = 0
    cnt = 0
    for i in range(len(A)):
        bi = A[i] + 100
        acc += bi
        cnt += 1
    return acc / cnt

In [7]:
%%time
f1(A)

Wall time: 1.06 s


599.811461

In [8]:
%time f1(A)

Wall time: 918 ms


599.811461

In [9]:
%%timeit
f1(A)

920 ms ± 53.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
# %time,%%time, %timeit

2. Напишите функцию, которая возвращает сумму всех чисел от 0 до x-1. Примените функцию к каждому элементу массива.

3. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [19]:
%load_ext line_profiler

In [7]:
import pandas as pd
import string

N = 2_000_000
df = pd.DataFrame(np.random.randn(N, 4), columns=[f"col{i}" for i in range(4)])
df["key"] = np.random.choice(list(string.ascii_letters.lower()), N, replace=True)
df.head(2)

,col0,col1,col2,col3,key
0,-0.076588,-1.521538,-1.190556,0.165123,r
1,0.236816,1.226909,-1.141244,-0.549996,u


In [ ]:
import numpy as np
from numba import njit

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.




def f2(A):
    acc = 0
    n = len(A)
    for i in range(n):
        acc += A[i]
 
    return acc / n + 100

%%time
f2(A)

%%time
A.mean() + 100

import numba

@njit
def f3(A):
    acc = 0
    n = len(A)
    for i in range(n):
        acc += A[i]
    return acc / n + 100

%%time
f3(A)

%%time
f3(A)

2. Напишите функцию, которая возвращает сумму всех чисел от 0 до x-1. Примените функцию к каждому элементу массива.

def g(x):
    return sum(range(x))

%%time
np.array([g(x) for x in A])

g_v = np.vectorize(g)

%%time
g_v(A)

A.dtype

g_v2 = numba.vectorize(["int32(int32)"])(g)

%%time
g_v2(A)

3. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

numba.__version__

%load_ext line_profiler

import pandas as pd
import string

N = 2_000_000
df = pd.DataFrame(np.random.randn(N, 4), columns=[f"col{i}" for i in range(4)])
df["key"] = np.random.choice(list(string.ascii_letters.lower()), N, replace=True)
df.head(2)

def h1(df):
    mask = []
    for _, row in df.iterrows():
        mask.append(row["key"] in {"a", "b", "c", "d", "e"})
    return df[mask]

%%time
h1(df.head(50000))

%lprun -f h1 h1(df.head(50000))

%%time
df[df["key"].isin({"a", "b", "c", "d", "e"})]

## Лабораторная работа 3

In [8]:
import numpy as np
import pandas as pd
from numba import jit, njit
import numba
from typing import Union


%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

## Измерение времени выполнения кода

Назовем полным описанием рецепта строку, полученную путем конкатенации названия и описания рецепта через пробел. Удалите строки для рецептов, которые были добавлены не в 2010 году.

Реализуйте несколько вариантов функции подсчета средней длины полного описания рецепта для рецептов, добавленных в 2010 году.

In [9]:
recipes = pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv')

In [10]:
# recipes['submitted'].dt.year
recp_2 = recipes[pd.to_datetime(recipes['submitted']).dt.year == 2010]
recp_2

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
52,just peachy cobbler,437637,70,1085867,2010-09-17,10.0,all i can say is yummmmmm . . . a simple to ma...,10.0
68,the heat spicy party mix,437219,95,1682162,2010-09-13,NaN,a spicy chex mix that will really warm your gu...,11.0
81,iowa state fair sweet dough caramel cinnamon ...,435816,80,17803,2010-08-24,29.0,this was the winning entry at the 2010 iowa st...,NaN
104,1 minute blueberries cream,428566,2,1375473,2010-06-04,4.0,i was craving blueberry tonight but wanted non...,NaN
146,2 2 2 diet mocha,416599,5,789314,2010-03-15,5.0,"while trying to come up with a satisfying ""sna...",7.0
...,...,...,...,...,...,...,...,...
29897,zoe s chicken tarragon,441211,40,76559,2010-11-04,12.0,from a good housekeeping at my hair salon. ha...,9.0
29907,zucchini and noodle slice,412518,60,423555,2010-02-10,21.0,"a yummy, tasty slice packed with vegies and ri...",13.0
29915,zucchini bread bread machine,409757,220,539686,2010-01-22,7.0,"originally from a packet of red star yeast, th...",NaN
29926,zucchini chip cupcakes,406686,35,628076,2010-01-04,9.0,this is a great tasting recipe to use up zucch...,14.0


1\.1 С использованием метода `DataFrame.iterrows` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - нахождение полного описания рецепта осуществляется внутри цикла по `iterrows` для каждой строки по отдельности.

In [11]:
def get_mean_len_A(df: pd.DataFrame) -> float:
    count = 0 
    for index, row in df.iterrows():
        length = len(str(row['name'])) + len(str(row['description']))
        count += length
    return count/df.shape[0]

In [14]:
%load_ext line_profiler
get_mean_len_A(recp_2)


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


264.501300390117

In [108]:
recp_2.shape[0]

1538

In [118]:
(recipes['name'].str.len() + recipes['description'].str.len()).mean()

229.43248119276984

1\.2. С использованием метода `DataFrame.apply` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - вызываете метод apply у таблицы, в качестве аргумента передаете функцию, которая возвращает полное описание для каждой строки;
    
    - считаете среднюю длину описаний, вызвав соответствующий метод серии.

In [235]:

def get_mean_len_B(df: pd.DataFrame):
    l = df.apply(lambda x: len(x['name'] + x['description']), axis=1).mean()
    return l 




In [237]:
%%timeit
get_mean_len_B(recp_2)


58.1 ms ± 993 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


1\.3. С использованием векторизованных методов серий `pd.Series`:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - при помощи векторизированных операций получаете столбец с полным описанием;
    
    - при помощи векторизированных операций получаете длины полного описания;
    
    - при помощи векторизированных операций получаете среднюю длину полных описаний. 

In [231]:
def get_mean_len_C(df: pd.DataFrame) -> float:
    return (df['name'] + df['description']).map(len).mean()

In [239]:
get_mean_len_C(recp_2)

264.501300390117

1.4 Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций при помощи магических команд `time` и `timeit`.

In [242]:
%%timeit
get_mean_len_A(recp_2)

219 ms ± 22.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [216]:
%%time
%%timeit
get_mean_len_B(recp_2)

62 ms ± 8.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Wall time: 5.05 s


In [234]:
%%timeit
get_mean_len_C(recp_2)

1.88 ms ± 68.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Анализ пошагового выполнения кода 

Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. 

In [76]:
import re


def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset = ["review"]).iterrows():
        recipe_id, review = row["recipe_id"], row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)

    word_reviews_count = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        review = row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [264]:
# reviews.dropna(subset=['review'])
# re.sub("[^A-Za-z\s]", "", str(reviews['review'])).split(" ")
word_reviews = {}
word_reviews['Last'] = []
word_reviews['Last'].append(245)
word_reviews

{'Last': [245]}

In [60]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)



2.1 Найдите узкие места в коде, проанализировав код функции по шагам, используя профайлер. Сохраните результаты работы профайлера в отдельную текстовую ячейку. Выпишите (словами), что в имеющемся коде реализовано неоптимально. 

In [77]:
%lprun -T lprof0 -f get_word_reviews_count get_word_reviews_count(reviews)



*** Profile printout saved to text file 'lprof0'. 


In [78]:
print(open('lprof0','r').read())

Timer unit: 1e-07 s

Total time: 122.262 s
File: <ipython-input-76-82f921a23349>
Function: get_word_reviews_count at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def get_word_reviews_count(df):
     5         1         27.0     27.0      0.0      word_reviews = {}
     6    126680  344740500.0   2721.3     28.2      for _, row in df.dropna(subset = ["review"]).iterrows():
     7    126679  123914418.0    978.2     10.1          recipe_id, review = row["recipe_id"], row["review"]
     8    126679   32013853.0    252.7      2.6          words = re.sub("[^A-Za-z\s]", "", review).split(" ")
     9   6918689   37573819.0      5.4      3.1          for word in words:
    10   6792010   56037369.0      8.3      4.6              if word not in word_reviews:
    11     93059     950747.0     10.2      0.1                  word_reviews[word] = []
    12   6792010   63468147.0      9.3      5.2              word_reviews[wo

Здесь легко можно понять что, две основные циклы с iterrows() занимают больше всего по времени 

2.2  Оптимизируйте функцию и добейтесь значительного (как минимум, в 5 раз) прироста в скорости выполнения. Для демонстрации результата измерьте скорость выполнения оригинальной функции и функции, написанной вами.

In [70]:
import re


def get_word_reviews_count(df):
    word_reviews = {}
    df1 = df.dropna(subset = ['review'])
    words = re.sub("[^A-Za-z\s]", "", str(df1['review'])).split(" ")
    
#     for _, row in df.dropna(subset = ["review"]).iterrows():
#         recipe_id, review = row["recipe_id"], row["review"]
#         words = re.sub("[^A-Za-z\s]", "", review).split(" ")
    for word in words:
        if word not in word_reviews:
            word_reviews[word] = 0
        word_reviews[word] += 1 #.append(recipe_id)
    return word_reviews
#     word_reviews_count = {}
#     for _, row in df.dropna(subset=["review"]).iterrows():
#         review = row["review"]
#         words = re.sub("[^A-Za-z\s]", "", review).split(" ")
#         for word in words:
#             word_reviews_count[word] = len(word_reviews[word])
#     return word_reviews_count

In [72]:
%lprun -T lprof0 -f get_word_reviews_count get_word_reviews_count(reviews)



*** Profile printout saved to text file 'lprof0'. 


In [75]:
print(open('lprof0','r').read())

Timer unit: 1e-07 s

Total time: 0.0527495 s
File: <ipython-input-70-d36700fac4a2>
Function: get_word_reviews_count at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def get_word_reviews_count(df):
     5         1         19.0     19.0      0.0      word_reviews = {}
     6         1     488191.0 488191.0     92.5      df1 = df.dropna(subset = ['review'])
     7         1      35145.0  35145.0      6.7      words = re.sub("[^A-Za-z\s]", "", str(df1['review'])).split(" ")
     8                                               
     9                                           #     for _, row in df.dropna(subset = ["review"]).iterrows():
    10                                           #         recipe_id, review = row["recipe_id"], row["review"]
    11                                           #         words = re.sub("[^A-Za-z\s]", "", review).split(" ")
    12       212       1000.0      4.7      0.2      for word

In [222]:
%%timeit
get_word_reviews_count(reviews) #было

49.3 s ± 4.27 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [67]:
%%timeit #стало
get_word_reviews_count(reviews) #Стало

38.2 ms ± 287 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Numba

В файле `rating_predictions.json` хранятся данные о рейтингах рецептов и прогнозных значениях рейтингов для этого рецепта, полученных при помощи модели машинного обучения. 

Напишите несколько версий функции (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от прогнозного значения рейтинга для данного рецепта. 


Замечание 1: в формуле MAPE под $A_t$ понимается рейтинг из отзыва $t$, под $F_t$ - прогнозное значения рейтинга отзыва $t$.

Замечание 2: в результате работы функций должно получиться одно число - MAPE для всего набора данных.

In [79]:
import json

In [82]:
with open('rating_predictions.json',
         'r') as f:
    book = json.load(f)
# book

In [161]:
a = 0
for i in book:
    #a += sum(abs((i['rating'] - i['prediction'])/i['rating'])) * 100 / len(i['rating'])
    a += abs((i['rating'] - i['prediction']) / i['rating']) * 100  / len(book) 
print(a)
    

13.3252655039899


In [83]:
def MAPE(book):
    return book

3\.1 Создайте два списка `A_list` и `F_list` на основе файла `rating_predictions.json`. Напишите функцию `mape_lists` без использования векторизованных операций и методов массивов `numpy` и без использования `numba` (проитерируйтесь по спискам и вычислите суммарное значение MAPE для всех элементов, а потом усредните результат).

Измерьте время выполнения данной функции на входных данных `A_list` и `F_list`. Временем, затрачиваемым на создание списков, можно пренебречь.
    

3\.2. Создайте массивы `numpy` `A_array` и `F_array` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numpy` с использованием векторизованных операций и методов массивов `numpy`.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`. Временем, затрачиваемым на создание массивов, можно пренебречь.

In [165]:
A_list = np.array([list(i.values())[1] for i in book])
F_list = np.array([list(i.values())[0] for i in book])

In [164]:
def MAPE(A_list, F_list):
    return sum(abs((A_list - F_list) / A_list)) *100 / len(A_list)
    

In [166]:
MAPE(rating, prediction)

11.949451393825532

3\.3. Создайте объекты `numba.typed.List` `A_typed` и `F_typed` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numba` без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`. 

Измерьте время выполнения данной функции на входных данных `A_typed` и `F_typed`. Временем, затрачиваемым на создание объектов `numba.typed.List`, можно пренебречь.

Измерьте время выполнения данной функции на входных данных `A_numpy` и `F_numpy`.

## Векторизация

Сайт-агрегатор устроил акцию: он дарит купоны на посещение ресторана тем пользователям, оставившим отзывы, идентификатор которых является _красивым числом_. Натуральное число называется _красивым_, если первая цифра числа совпадает с последней цифрой числа. 



4\.1 Напишите функцию `is_pretty`, которая для каждого идентификатора пользователя из файла определяет, получит ли он подарок. Запрещается преобразовывать идентификатор пользователя к строке. Подтвердите корректность реализации, продемонстрировав примеры.

In [ ]:
ids = reviews["recipe_id"].values

In [ ]:
def is_pretty(n: int) -> bool:
    pass

4\.2 Посчитайте с помощью функции `is_pretty` количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.

4\.3. При помощи `numpy` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


4\.4. При помощи `numba` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.
